Cargas librerías

In [26]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
from mlxtend.preprocessing import TransactionEncoder 
import re
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

In [27]:
#### conectar_base_de_Datos
conn=sql.connect('db_movies')
cur=conn.cursor()

In [28]:
#### ver tablas disponibles en base de datos ###

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',),
 ('reco',)]

Sistema de recomendación basado en contenido un solo producto - Manual

In [29]:
#Se crea tabla master
movies=pd.read_sql('select * from movies_final', conn )

In [30]:
#Se realiza el .ifno para observar detalles de la tabla
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2441 entries, 0 to 2440
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  2441 non-null   int64 
 1   title    2441 non-null   object
 2   genres   2441 non-null   object
dtypes: int64(1), object(2)
memory usage: 57.3+ KB


In [31]:
# se convierte la columna genres en variables dummies
genres=movies['genres'].str.split('|')

te = TransactionEncoder()

genres = te.fit_transform(genres)

genres = pd.DataFrame(genres, columns = te.columns_)

movies = pd.concat([movies, genres], axis=1).drop(columns=["genres"])

movies.iloc[:,2:]=movies.iloc[:,2:].astype(int)

In [32]:

#Separar año del titulo
regex = r"\([0-9]+\)"
movies["year"]=movies.title.apply(lambda x:re.search(regex, x).group(0).replace("(","").replace(")",""))
movies["title"]=movies.title.apply(lambda x:re.sub(regex, "",x))

In [33]:
movies['year']=movies.year.astype('int')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2441 entries, 0 to 2440
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieId      2441 non-null   int64 
 1   title        2441 non-null   object
 2   Action       2441 non-null   int32 
 3   Adventure    2441 non-null   int32 
 4   Animation    2441 non-null   int32 
 5   Children     2441 non-null   int32 
 6   Comedy       2441 non-null   int32 
 7   Crime        2441 non-null   int32 
 8   Documentary  2441 non-null   int32 
 9   Drama        2441 non-null   int32 
 10  Fantasy      2441 non-null   int32 
 11  Film-Noir    2441 non-null   int32 
 12  Horror       2441 non-null   int32 
 13  IMAX         2441 non-null   int32 
 14  Musical      2441 non-null   int32 
 15  Mystery      2441 non-null   int32 
 16  Romance      2441 non-null   int32 
 17  Sci-Fi       2441 non-null   int32 
 18  Thriller     2441 non-null   int32 
 19  War          2441 non-null 

In [34]:
#Escalonar year
movies['year']=movies.year.astype('int')
sc=MinMaxScaler()
movies[["year"]]=sc.fit_transform(movies[['year']])

In [35]:
#se agrga la columna titulo a la tabla genres y se escalona
genres1= pd.get_dummies(movies,columns =['title'])
genres1 = genres1.drop(columns=['movieId','year'])

In [36]:
def recomendacion(movie = list(movies['title'])):
     
    ind_movie=movies[movies['title']==movie].index.values.astype(int)[0]   
    similar_movies = genres1.corrwith(genres1.iloc[ind_movie,:],axis=1)
    similar_movies = similar_movies.sort_values(ascending=False)
    top_similar_movies=similar_movies.to_frame(name="correlación").iloc[0:10,]
    top_similar_movies['title']=movies["title"]
    
    return top_similar_movies


print(interact(recomendacion))

interactive(children=(Dropdown(description='movie', options=('Toy Story ', 'Jumanji ', 'Grumpier Old Men ', 'F…

<function recomendacion at 0x0000023393684040>


Sistema de recomendación basado en contenido KNN un solo producto visto

In [46]:
#Entrenar modelo
model = neighbors.NearestNeighbors(n_neighbors=10, metric='cosine')
model.fit(genres)
dist, idlist = model.kneighbors(genres)

distancias=pd.DataFrame(dist)
id_list=pd.DataFrame(idlist)


In [48]:

def BookRecommender(movie_name = list(movies['title'].value_counts().index)):
    movie_list_name = []
    movie_id = movies[movies['title'] == movie_name].index
    movie_id = movie_id[0]
    for newid in idlist[movie_id]:
        movie_list_name.append(movies.loc[newid].title)
    return movie_list_name



print(interact(BookRecommender))

interactive(children=(Dropdown(description='movie_name', options=('King Kong ', 'Cape Fear ', 'Parent Trap, Th…

<function BookRecommender at 0x00000233F3E458B0>


Sistema de recomendación basado en contenido KNN Con base en todo lo visto por el usuario

In [39]:
#Se crea tabla con los usuarios
usuarios=pd.read_sql('select distinct (userId) as userId from ratings_final',conn)

In [40]:
#Se crea tabla donde se encuentran los rating para viasualizarlos
full=pd.read_sql('select * from ratings_final', conn )

In [49]:
def recomendar(user_id=list(usuarios['userId'].value_counts().index)):
    
    ###seleccionar solo los ratings del usuario seleccionado
    ratings=pd.read_sql('select *from ratings_final where userId=:user',conn, params={'user':user_id})
    ###convertir ratings del usuario a array
    l_movies_r=ratings['movieId'].to_numpy()
    
    ###agregar la columna de isbn y titulo del libro a dummie para filtrar y mostrar nombre
    genres1[['movieId','title']]=movies[['movieId','title']]
    movies_r=genres1[genres1['movieId'].isin(l_movies_r)]
    movies_r=movies_r.drop(columns=['movieId','title'])
    movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    centroide=movies_r.groupby("indice").mean()
    
    
    movies_nr=genres1[~genres1['movieId'].isin(l_movies_r)]
    movies_nr=movies_nr.drop(columns=['movieId','title'])
    model=neighbors.NearestNeighbors(n_neighbors=10, metric='cosine')
    model.fit(movies_nr)
    dist, idlist = model.kneighbors(centroide)
    
    ids=idlist[0]
    recomend_b=movies.loc[ids][['title','movieId']]
    vistos=movies[movies['movieId'].isin(l_movies_r)][['title','movieId']]
    
    return recomend_b


recomendar(22)


print(interact(recomendar))

interactive(children=(Dropdown(description='user_id', options=(1, 401, 416, 415, 414, 413, 412, 411, 410, 409,…

<function recomendar at 0x00000233F3E45820>


Sistema de recomendación filtro colaborativo basado en usuario

In [42]:
movies_final=pd.read_sql('select * from movies_final', conn)

In [43]:
full.to_csv('././rating.csv', index=False)
movies_final.to_csv('././movies_final.csv', index=False)

In [44]:
ratings=pd.read_sql('select * from ratings_final', conn)
pd.read_sql('select avg(rating) from ratings_final', conn) ## promedio de ratings
###### leer datos desde tabla de pandas
reader = Reader(rating_scale=(0, 5))

###las columnas deben estar en orden estándar: user item rating
data   = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

### los datos se pueden cargar desde un dataframe al formato que reciben las funciones de surprise

#####Existen varios modelos 

models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()] 
results = {}

###knnBasiscs: calcula el rating ponderando por distancia con usuario/Items
###KnnWith means: en la ponderación se resta la media del rating, y al final se suma la media general
####KnnwithZscores: estandariza el rating restando media y dividiendo por desviación 
####Knnbaseline: calculan el desvío de cada calificación con respecto al promedio y con base en esos calculan la ponderación



for model in models:
 
    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)  
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result


performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')


param_grid = { 'sim_options' : {'name': ['msd','cosine'], \
                                'min_support': [5], \
                                'user_based': [False, True]}
             }

gridsearchKNNWithMeans = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], \
                                      cv=2, n_jobs=2)
                                    
gridsearchKNNWithMeans.fit(data)


gridsearchKNNWithMeans.best_params["rmse"]
gridsearchKNNWithMeans.best_score["rmse"]
gs_model=gridsearchKNNWithMeans.best_estimator['rmse'] ### mejor estimador de gridsearch


################# Realizar predicciones

trainset = data.build_full_trainset() ### esta función convierte todos los datos en entrnamiento
model=gs_model.fit(trainset) ## se entrena sobre todos los datos posibles


predset = trainset.build_anti_testset() ### crea una tabla con todos los usuarios y los libros que no han leido
#### en la columna de rating pone el promedio de todos los rating, en caso de que no pueda calcularlo para un item-usuario

predictions = model.test(predset) ### función muy pesada, hace las predicciones de rating para todos los libros que no hay leido un usuario
### la funcion test recibe un test set constriuido con build_test method, o el que genera crosvalidate

predictions_df = pd.DataFrame(predictions) ### esta tabla se puede llevar a una base donde estarán todas las predicciones
predictions_df.shape
predictions_df.head()
predictions_df['r_ui'].unique() ### promedio de ratings
predictions_df.sort_values(by='est',ascending=False)

####### la predicción se puede hacer para un libro puntual
model.predict(uid='31226', iid='0373825013',r_ui='2.42')

##### funcion para recomendar los 10 libros con mejores predicciones y llevar base de datos para consultar resto de información
def recomendaciones(user_id,n_recomend=10):
    
    predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                    sort_values(by="est", ascending = False).head(n_recomend)

    recomendados = predictions_userID[['iid','est']]
    recomendados.to_sql('reco',conn,if_exists="replace")
    
    recomendados=pd.read_sql('''select a.*, b.title 
                             from reco a left join movies_final b
                             on a.iid=b.movieId ''', conn)

    return(recomendados)


 


Computing the msd similarity matrix...
Done computing similarity matrix.


In [45]:
us1=recomendaciones(user_id=22,n_recomend=10)
us1

,index,iid,est,title
0,43097,1178,3.578874,Paths of Glory (1957)
1,43053,1411,3.524280,Hamlet (1996)
2,41139,720,3.521931,Wallace & Gromit: The Best of Aardman Animatio...
3,42498,1204,3.460435,Lawrence of Arabia (1962)
4,42454,8132,3.442947,Gladiator (1992)
5,42394,3836,3.408695,Kelly's Heroes (1970)
6,41211,1283,3.402682,High Noon (1952)
7,41878,71899,3.387552,Mary and Max (2009)
8,42391,3451,3.382355,Guess Who's Coming to Dinner (1967)
9,42518,1104,3.381802,"Streetcar Named Desire, A (1951)"
